In [3]:
import pandas as pd
import pointblank as pb
import function.data as cd
import datetime

In [4]:
endpoint = "https://data.cityofchicago.org/resource/ijzp-q8t2"

start = datetime.datetime(2023,1,1,0,0,0)
end = datetime.datetime(2025,5,1,0,0,0)

In [5]:

df = cd.backfill_chicago_data(endpoint = endpoint, 
                                start = start, 
                                end = end, 
                                offset = 24 * 30,
                                limit = 100000)

In [6]:

df["updated_on"] = pd.to_datetime(df["updated_on"])
df["x_coordinate"] = df["x_coordinate"].astype(float)
df["y_coordinate"] = df["y_coordinate"].astype(float)
df["latitude"] = df["latitude"].astype(float)
df["longitude"] = df["longitude"].astype(float)
df["year"]= df["year"].astype(int)

In [7]:

df.head()

,id,case_number,datetime,block,iucr,primary_type,description,location_description,arrest,domestic,...,district,ward,community_area,fbi_code,x_coordinate,y_coordinate,year,updated_on,latitude,longitude
0,13140855,JG341458,2023-01-01,082XX S JEFFERY BLVD,1754,OFFENSE INVOLVING CHILDREN,AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...,APARTMENT,False,True,...,004,8,46,02,1190953.0,1850848.0,2023,2023-09-24 15:41:26,41.745739,-87.575883
1,13180096,JG387858,2023-01-01,075XX S WOLCOTT AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,006,17,71,11,1164996.0,1854651.0,2023,2023-08-20 15:40:56,41.756762,-87.670887
2,13168471,JG374193,2023-01-01,013XX W HARRISON ST,0460,BATTERY,SIMPLE,SCHOOL - PUBLIC GROUNDS,False,False,...,012,34,28,08B,1167465.0,1897475.0,2023,2023-08-19 15:40:26,41.874223,-87.660610
3,13078152,JG267031,2023-01-01,101XX S BEVERLY AVE,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,022,21,73,11,1168800.0,1837525.0,2023,2023-08-19 15:40:26,41.709685,-87.657439
4,13120699,JG314178,2023-01-01,063XX N FAIRFIELD AVE,1544,SEX OFFENSE,SEXUAL EXPLOITATION OF A CHILD,OTHER (SPECIFY),False,False,...,024,50,2,17,1156847.0,1941985.0,2023,2023-08-19 15:40:26,41.996584,-87.698384


In [8]:

df.dtypes

id                              object
case_number                     object
datetime                datetime64[ns]
block                           object
iucr                            object
primary_type                    object
description                     object
location_description            object
arrest                            bool
domestic                          bool
beat                            object
district                        object
ward                            object
community_area                  object
fbi_code                        object
x_coordinate                   float64
y_coordinate                   float64
year                             int64
updated_on              datetime64[ns]
latitude                       float64
longitude                      float64
dtype: object

In [9]:
schema = pb.Schema(
    columns=[
        ("id", "object"),
        ("case_number", "object"),
        ("datetime", "datetime64[ns]"),   
        ("block", "object"),
        ("iucr", "object"),
        ("primary_type", "object"),
        ("description", "object"),
        ("location_description", "object"),
        ("arrest", "bool"),
        ("domestic", "bool"),
        ("beat", "object"),
        ("district", "object"),
        ("ward", "object"),
        ("community_area", "object"),
        ("fbi_code", "object"),
        ("x_coordinate", "float64"),
        ("y_coordinate", "float64"),
        ("year", "int64"),
        ("updated_on", "datetime64[ns]"),
        ("latitude", "float64"),
        ("longitude", "float64")
    ]
)

In [10]:

validation = (
            pb.Validate(data = df,
            tbl_name= "Chicago Crime Data",
            label = "Chicago Crime Data",
            thresholds=pb.Thresholds(warning= 0, error= 0, critical= 0))
            .col_schema_match(schema=schema)
            .col_count_match(count=len(schema.columns)) 
            .col_vals_not_null(columns= ["case_number", "datetime","year"])
            .rows_distinct() 
            .interrogate()
        )

In [11]:

validation

Validate(data=              id case_number            datetime                      block  \
0       13140855    JG341458 2023-01-01 00:00:00       082XX S JEFFERY BLVD   
1       13180096    JG387858 2023-01-01 00:00:00        075XX S WOLCOTT AVE   
2       13168471    JG374193 2023-01-01 00:00:00        013XX W HARRISON ST   
3       13078152    JG267031 2023-01-01 00:00:00        101XX S BEVERLY AVE   
4       13120699    JG314178 2023-01-01 00:00:00      063XX N FAIRFIELD AVE   
...          ...         ...                 ...                        ...   
593718  13822428    JJ238259 2025-04-30 23:00:00       026XX N LAKEWOOD AVE   
593719  13822457    JJ238254 2025-04-30 23:00:00  035XX S COTTAGE GROVE AVE   
593720  13822270    JJ238117 2025-04-30 23:00:00           001XX E 113TH ST   
593721  13822800    JJ238725 2025-04-30 23:00:00          081XX S ELLIS AVE   
593722  13828681    JJ246159 2025-04-30 23:00:00       020XX S MICHIGAN AVE   

        iucr                primary_type  \
0       1754  OFFENSE INVOLVING CHILDREN   
1       1153          DECEPTIVE PRACTICE   
2       0460                     BATTERY   
3       1153          DECEPTIVE PRACTICE   
4       1544                 SEX OFFENSE   
...      ...                         ...   
593718  1330           CRIMINAL TRESPASS   
593719  0910         MOTOR VEHICLE THEFT   
593720  0610                    BURGLARY   
593721  1792                  KIDNAPPING   
593722  0920         MOTOR VEHICLE THEFT   

                                              description  \
0       AGGRAVATED SEXUAL ASSAULT OF CHILD BY FAMILY M...   
1                     FINANCIAL IDENTITY THEFT OVER $ 300   
2                                                  SIMPLE   
3                     FINANCIAL IDENTITY THEFT OVER $ 300   
4                          SEXUAL EXPLOITATION OF A CHILD   
...                                                   ...   
593718                                            TO LAND   
593719                                         AUTOMOBILE   
593720                                     FORCIBLE ENTRY   
593721                         CHILD ABDUCTION / STRANGER   
593722                               ATTEMPT - AUTOMOBILE   

                          location_description  arrest  domestic  ...  \
0                                    APARTMENT   False      True  ...   
1                                    RESIDENCE   False     False  ...   
2                      SCHOOL - PUBLIC GROUNDS   False     False  ...   
3                                    RESIDENCE   False     False  ...   
4                              OTHER (SPECIFY)   False     False  ...   
...                                        ...     ...       ...  ...   
593718                               RESIDENCE   False     False  ...   
593719                       VACANT LOT / LAND   False     False  ...   
593720                               APARTMENT   False     False  ...   
593721                               APARTMENT   False      True  ...   
593722  PARKING LOT / GARAGE (NON RESIDENTIAL)   False     False  ...   

       district ward community_area fbi_code x_coordinate  y_coordinate  year  \
0           004    8             46       02    1190953.0     1850848.0  2023   
1           006   17             71       11    1164996.0     1854651.0  2023   
2           012   34             28      08B    1167465.0     1897475.0  2023   
3           022   21             73       11    1168800.0     1837525.0  2023   
4           024   50              2       17    1156847.0     1941985.0  2023   
...         ...  ...            ...      ...          ...           ...   ...   
593718      019   32              7       26    1167183.0     1917653.0  2025   
593719      002    4             36       07    1181368.0     1881512.0  2025   
593720      005    9             49       05    1179403.0     1830076.0  2025   
593721      006    8             44       26    1184311.0     1851188.0  2025   
593722 

In [12]:
validation.all_passed()

True

In [13]:

df.to_csv("data/chicago_crime_2023_2025.csv", index = False)